In [1]:
import sys
sys.path.append("../")
import geopandas as gpd
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score
import streamlit as st
import pandas as pd
import sys
from src.graph_layering.data_processing import Normalizer
from src.lightning.hetero_gnn_module import HeteroGNNModule
import os
import pandas as pd
import torch
import geopandas as gpd
import numpy as np
from sklearn.metrics import f1_score
from streamlit_folium import st_folium
from src.organized_datasets_creation.utils.nominatim import (
    convert_nominatim_name_to_filename,
    resolve_nominatim_city_name,
)
from shapely.geometry import Point
import joblib
from sklearn.preprocessing import StandardScaler


In [2]:
import os
os.path.exists("../data/")

True

In [6]:
# ACCIDENTS_LOCATION = "../data/downstream_tasks/accidents_prediction/accidents.csv"
GRAPH_DATA_DICT_PATH = "../data/results_showcase/zabka_shops/tabular_data_zabka.pkl"
MODEL_PATH = "../data/results_showcase/zabka_shops/model_zabka.pkl"
ORGANIZED_HEXES_LOCATION = "../data/organized-hexes"


def load_graph_data_and_model():
    data = joblib.load(GRAPH_DATA_DICT_PATH)
    model = joblib.load(MODEL_PATH)
    return data, model

In [7]:
data, model = load_graph_data_and_model()

In [8]:
data['Wrocław']['X'].head()

,shop_convenience,amenity_parking,building_apartments,amenity_parking_entrance,building_retail,shop_supermarket,amenity_parcel_locker,leisure_playground,amenity_pharmacy,landuse_grass,...,amenity_e-scooter_parking,sport_swimming;table_tennis;fitness;multi,sport_vault_jump,healthcare_first_aid,leisure_parking,shop_Shag Shop,building_palace,shop_tools,building_power,natural_tree_group
region_id,,,,,,,,,,,,,,,,,,,,,
0,0,2,5,6,0,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,13,6,0,1,0,5,3,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,4,1,0,1,0,1,1,1,6,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data.keys()

dict_keys(['Kraków', 'Poznań', 'Szczecin', 'Wrocław', 'Warszawa'])

In [10]:
'hex_id' in data['Wrocław']['X'].columns

False

dodanie kolumny `hex_id`

In [8]:
for City, city in [('Kraków', 'krakow'), ('Poznań', 'poznan'), ('Szczecin', 'szczecin'), ('Wrocław', 'wroclaw'), ('Warszawa', 'warszawa')]:
    df_region_id = pd.read_parquet(f"{ORGANIZED_HEXES_LOCATION}/{city}/2022/h9/count-embedder/dataset.parquet")['region_id']
    data[City]['X'] = data[City]['X'].join(df_region_id).rename(columns={'region_id': 'hex_id'})

In [9]:
'hex_id' in data['Wrocław']['X'].columns

True

In [10]:
import pickle

GRAPH_DATA_DICT_PATH = f'{GRAPH_DATA_DICT_PATH[:-4]}_hex_id.pkl'

with open(GRAPH_DATA_DICT_PATH, 'wb') as file:
    pickle.dump(data, file)


_____

sprawdzenie

In [3]:
# ACCIDENTS_LOCATION = "../data/downstream_tasks/accidents_prediction/accidents.csv"
GRAPH_DATA_DICT_PATH = "../data/results_showcase/zabka_shops/tabular_data_zabka_hex_id.pkl" # już podmieniona ścieżka 
MODEL_PATH = "../data/results_showcase/zabka_shops/model_zabka.pkl"
ORGANIZED_HEXES_LOCATION = "../data/organized-hexes"


def load_graph_data_and_model():
    data = joblib.load(GRAPH_DATA_DICT_PATH)
    model = joblib.load(MODEL_PATH)
    return data, model

In [4]:
data_with_hex_id, model = load_graph_data_and_model()
'hex_id' in data_with_hex_id['Wrocław']['X'].columns

True

In [11]:
data_with_hex_id['Wrocław']['X']

,shop_convenience,amenity_parking,building_apartments,amenity_parking_entrance,building_retail,shop_supermarket,amenity_parcel_locker,leisure_playground,amenity_pharmacy,landuse_grass,...,sport_swimming;table_tennis;fitness;multi,sport_vault_jump,healthcare_first_aid,leisure_parking,shop_Shag Shop,building_palace,shop_tools,building_power,natural_tree_group,hex_id
region_id,,,,,,,,,,,,,,,,,,,,,
0,0,2,5,6,0,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,891e204e4a7ffff
1,2,13,6,0,1,0,5,3,1,1,...,0,0,0,0,0,0,0,0,0,891e204239bffff
2,2,4,1,0,1,0,1,1,1,6,...,0,0,0,0,0,0,0,0,0,891e2041837ffff
3,0,1,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,891e2047223ffff
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,891e2046657ffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,891e2041137ffff
3164,0,3,0,0,0,0,0,0,0,21,...,0,0,0,0,0,0,0,0,0,891e2042273ffff
3165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,891e20430d7ffff


In [12]:
city_value = 'Wrocław'

In [13]:
for City in ['Kraków', 'Poznań', 'Szczecin', 'Wrocław', 'Warszawa']:
    data[City]['X'] = data_with_hex_id[City]['X'].drop(columns=['hex_id'])

In [15]:
data["Wrocław"]['X']

,shop_convenience,amenity_parking,building_apartments,amenity_parking_entrance,building_retail,shop_supermarket,amenity_parcel_locker,leisure_playground,amenity_pharmacy,landuse_grass,...,amenity_e-scooter_parking,sport_swimming;table_tennis;fitness;multi,sport_vault_jump,healthcare_first_aid,leisure_parking,shop_Shag Shop,building_palace,shop_tools,building_power,natural_tree_group
region_id,,,,,,,,,,,,,,,,,,,,,
0,0,2,5,6,0,0,2,3,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,13,6,0,1,0,5,3,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,4,1,0,1,0,1,1,1,6,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3164,0,3,0,0,0,0,0,0,0,21,...,0,0,0,0,0,0,0,0,0,0
3165,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
folds = [
    ("Wrocław", "Kraków"),
    ("Kraków", "Poznań"),
    ("Poznań", "Szczecin"),
    ("Szczecin", "Warszawa"),
    ("Warszawa", "Wrocław"),
]
scaler = StandardScaler()
X = pd.concat(
[
    m["X"]
    for key, m in data.items()
    if key != city_value
]
).to_numpy()
y = (
pd.concat(
    [
        m["y"]
        for key, m in data.items()
        if key != city_value
    ]
)
.to_numpy()
.ravel()
)

X = scaler.fit_transform(X)
test_X = data[city_value]["X"].to_numpy()
test_X = scaler.transform(test_X)
test_y = data[city_value]["y"].to_numpy().ravel()
y_pred = model.predict(test_X)
y_proba = model.predict_proba(test_X)[:, 1]

auc = roc_auc_score(test_y, y_proba, average="micro")
accuracy = (y_pred == test_y).mean()
f1 = f1_score(
    test_y,
    y_pred,
    pos_label=1,
    average="binary",
)
f1

0.7599451303155007

In [12]:
GRAPH_DATA_DICT_PATH = "../data/results_showcase/accidents/data.pkl"
MODEL_PATH = "../data/results_showcase/accidents/model.ckpt"
ORGANIZED_HEXES_LOCATION = "../data/organized-hexes"


def load_graph_data_and_model():
    data = pd.read_pickle(GRAPH_DATA_DICT_PATH)

    model = HeteroGNNModule.load_from_checkpoint(
        MODEL_PATH, hetero_data=list(data.values())[3], map_location=torch.device("cpu")
    )
    return data, model


data1, model1 = load_graph_data_and_model()
data1

{'Wrocław, Poland': CityHeteroData(
   hex={
     x=[3168, 0],
     y=[3168],
   },
   osmnx_node={ x=[71641, 22] },
   (hex, connected_to, hex)={ edge_index=[2, 9206] },
   (osmnx_node, connected_to, osmnx_node)={
     edge_index=[2, 124386],
     edge_attr=[124386, 44],
   },
   (osmnx_node, connected_to, hex)={
     edge_index=[2, 71641],
     edge_attr=[71641, 0],
   }
 ),
 'Szczecin, Poland': CityHeteroData(
   hex={
     x=[3534, 0],
     y=[3534],
   },
   osmnx_node={ x=[64894, 22] },
   (hex, connected_to, hex)={ edge_index=[2, 10163] },
   (osmnx_node, connected_to, osmnx_node)={
     edge_index=[2, 106588],
     edge_attr=[106588, 44],
   },
   (osmnx_node, connected_to, hex)={
     edge_index=[2, 64894],
     edge_attr=[64894, 0],
   }
 ),
 'Poznań, Poland': CityHeteroData(
   hex={
     x=[2945, 0],
     y=[2945],
   },
   osmnx_node={ x=[60082, 22] },
   (hex, connected_to, hex)={ edge_index=[2, 8486] },
   (osmnx_node, connected_to, osmnx_node)={
     edge_index=[2, 9865